In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()

  Activating environment at `~/study/16715robotsim/project/Project.toml`


In [2]:
using LinearAlgebra
using ForwardDiff
using OrdinaryDiffEq
using Ipopt
using MathOptInterface
const MOI = MathOptInterface
using Test
using Test
using Plots
using Convex, SCS
using JLD

## System properties

In [3]:
# simulation step size
Δt = 0.05
const g = 9.81

m = 1
μ = 0.5

w = 1.0 # width of the box
h = 0.8 # height of the box
M = [m 0 0; 0 m 0; 0 0 m*((h/2)^2+(w/2)^2)/3]

# set of contact modes
modes = [0 0 0 0; # both free
        1 0 0 0; # sticking
        1 0 -1 0; # left-slide
        1 0 1 0; # right-slide
        0 1 0 0;
        0 1 0 -1;
        0 1 0 1;
        1 1 0 0;
        1 1 -1 -1;
        1 1 1 1]

n_contacts = 2


2

In [4]:
tol_c = 1e-5

1.0e-5

## Simulation functions

In [5]:
function R_2D(θ)
    R = [cos(θ) -sin(θ); sin(θ) cos(θ)]
    return R
end

R_2D (generic function with 1 method)

In [6]:
# constraints/contacts
function compute_a(q)
    y = q[2]
    θ = q[3]
    a1 = y - 0.5*h*cos(θ) - 0.5*w*sin(θ)
    a2 = y - 0.5*h*cos(θ) + 0.5*w*sin(θ)
    return [a1; a2]
end

function compute_a_t(q)
    p1 = R_2D(q[3])*[-w/2; -h/2] 
    p2 = R_2D(q[3])*[w/2; -h/2] 
    at1 = p1[1] + q[1]
    at2 = p2[1] + q[1]
    return [at1; at2]
end

# constraints jacobian
function compute_A(q)
    θ = q[3]
    A = [0 1 0.5*h*sin(θ)-0.5*w*cos(θ); 0 1 0.5*h*sin(θ)+0.5*w*cos(θ)]
    return A
end

function compute_dA(q, dq)
    dA = reshape(ForwardDiff.jacobian(compute_A, q)*dq, n_contacts, 3)
    return dA
end

function compute_A_tangent(q)
    A_t = ForwardDiff.jacobian(compute_a_t,q)
    return A_t
end

function compute_dA_tangent(q, dq)
    dA_t = reshape(ForwardDiff.jacobian(compute_A_tangent, q)*dq, n_contacts, 3)
    return dA_t
end

compute_dA_tangent (generic function with 1 method)

In [7]:
function contact_mode_constraints(x, contactMode)
    
    q = x[1:3]
    dq = x[4:6]
    
    cs_mode = contactMode[1:n_contacts] .== 1
    ss_mode = contactMode[n_contacts+1:end]
    
    A = compute_A(q)
    A_t = compute_A_tangent(q)
    dA = compute_dA(q, dq)
    dA_t = compute_dA_tangent(q,dq)
    A = A[cs_mode,:]
    A_t = A_t[cs_mode,:]
    dA = dA[cs_mode,:]
    dA_t = dA_t[cs_mode,:]
    
    ss_active = ss_mode[cs_mode]
    
    A_all = zeros(0,3)
    A_all_f = zeros(0,3)
    dA_all = zeros(0,3)

    for k = 1:length(ss_active)
        ss = ss_active[k]
        if ss == 0
            A_all_f = [A_all_f; A[k,:]'; A_t[k,:]']
            dA_all = [dA_all; dA[k,:]'; dA_t[k,:]']
            A_all = [A_all; A[k,:]'; A_t[k,:]']
        else
            A_all_f = [A_all_f; A[k,:]'-ss*μ*A_t[k,:]']
            dA_all = [dA_all; dA[k,:]']
            A_all = [A_all; A[k,:]']
        end
    end
    
    return A_all_f, A_all, dA_all
end

contact_mode_constraints (generic function with 1 method)

In [8]:
function contact_force_constraints(λ, contactMode)
    # c > 0
    cs_mode = contactMode[1:n_contacts] .== 1
    ss_mode = contactMode[n_contacts+1:end]
    
    ss_active = ss_mode[cs_mode]
    
    i = 1
    ic = 1
    c = zeros(sum(ss_active.==0)*3 + sum(ss_active.!=0))
    for k = 1:length(ss_active)
        ss = ss_active[k]
        if ss == 0
            c[ic:ic+2] = [-λ[i]; -μ*λ[i] - λ[i+1]; -μ*λ[i] + λ[i+1]]
            ic += 3
            i += 2
        else
            c[ic] = -λ[i]
            i += 1
            ic += 1
        end
    end
    return c
end

contact_force_constraints (generic function with 1 method)

In [9]:
function sliding_velocity_constraints(x, contactMode)
    # reture A_eq, A_geq, A_eq*dq = 0, A_geq*dq >= 0
    q = x[1:3]
    dq = x[4:6]
    
    cs_mode = contactMode[1:n_contacts] .== 1
    ss_mode = contactMode[n_contacts+1:end]
    
    A = compute_A(q)
    A_t = compute_A_tangent(q)
    dA = compute_dA(q, dq)
    dA_t = compute_dA_tangent(q,dq)
    A = A[cs_mode,:]
    A_t = A_t[cs_mode,:]
    dA = dA[cs_mode,:]
    dA_t = dA_t[cs_mode,:]
    
    ss_active = ss_mode[cs_mode]
    
    AA_eq = zeros(0,3)
    AA_geq = zeros(0,3)
    dAA_eq = zeros(0,3)
    dAA_geq = zeros(0,3)

    for k = 1:length(ss_active)
        ss = ss_active[k]
        if ss == 0
            AA_eq = [AA_eq; A[k,:]'; A_t[k,:]']
            dAA_eq = [dAA_eq; dA[k,:]'; dA_t[k,:]']
        else
            AA_eq = [AA_eq; A[k,:]']
            dAA_eq = [dAA_eq; dA[k,:]']
            AA_geq = [AA_geq; ss*A_t[k,:]']
            dAA_geq = [dAA_geq; ss*dA_t[k,:]']
        end
    end
    
    return AA_eq, AA_geq, dAA_eq, dAA_geq
end

sliding_velocity_constraints (generic function with 1 method)

In [10]:
function solveEOM(x, contactMode, controller)
    # contactMode: bool vector, indicates which constraints are active
    q = x[1:3]
    dq = x[4:6]
    
    A_f, A, dA = contact_mode_constraints(x, contactMode)
    
    # compute EOM matrices
    N = [0; g; 0]
    C = zeros(3,3)
    if typeof(controller) == typeof(zeros(3))
        Y = controller
    else
        Y = controller(x, contactMode)
    end
    
    #
    blockMat = [M A_f'; A zeros(size(A,1),size(A_f',2))] 

    b = [Y-N-C*dq; -dA*dq]
    
    #z = blockMat\b
    #println(blockMat)
    if rank(blockMat) < length(b)
#         z =pinv(blockMat)*b
        H = [zeros(3,size(blockMat,2)); zeros(size(blockMat,1)-3,3) 1e-6*I]
        z =(blockMat.+H)\b
    else
        z = blockMat\b
    end
    
    ddq = z[1:3]
    if (sum(contactMode[1:n_contacts])>=1)
        λ = z[4:end]
    else
        λ = []
    end
    
    return ddq, λ
end

solveEOM (generic function with 1 method)

In [11]:
function computeResetMap(x, contactMode)
    q = x[1:3]
    dq = x[4:6]

    A_f, A, dA = contact_mode_constraints(x, contactMode)
    
    c = size(A, 1)
    #
    blockMat = [M A_f'; A zeros(size(A,1),size(A_f',2))] 
    
    if rank(blockMat) < 3+c
        z = pinv(blockMat)*[M*dq; zeros(c)]
    else
        z = blockMat\[M*dq; zeros(c)]
    end
        
    dq_p = z[1:3]
    p_hat = z[4:end]
    return dq_p, p_hat
end

computeResetMap (generic function with 1 method)

In [12]:
function compute_FA(x, u_control)
    
    q = x[1:3]
    dq = x[4:6]
    
    a = compute_a(q)
    
    active_cs = abs.(a) .< tol_c
    inactive_cs = abs.(a) .> tol_c
    
    if sum(active_cs) == 0
        return zeros(4)
    end
    
    possibleModes = zeros(Bool, 0, size(modes,2))
    
    contactMode = zeros(n_contacts*2)
    
    for k = 1:size(modes,1)
        m_cs = modes[k, 1:n_contacts].==1
        if length(findall(z->z==true, m_cs[inactive_cs])) == 0
            possibleModes = [possibleModes; modes[k, :]']
        end
    end

    max_cons = 0
    
    
    for kk = 1:size(possibleModes, 1)      
        
        m = possibleModes[kk,:]
        
        separate_cs = (m[1:n_contacts].!=1) .& active_cs
        _, A_separate, dA_separate = contact_mode_constraints(x, [separate_cs; ones(n_contacts)]) 

        ddq, λ = solveEOM(x, m, u_control)
        
        c_λ = contact_force_constraints(λ, m)
        
        if all(c_λ.>=0)
        
            As_eq, As_geq, dAs_eq, dAs_geq = sliding_velocity_constraints(x, m)

            sep_vel_cond = ((A_separate*dq).>0) .| ((A_separate*ddq .+ dA_separate*dq).>=0)
            maintain_vel_cond = all(abs.(As_eq*dq).<tol_c) & all((As_geq*dq).>tol_c)

            if ~maintain_vel_cond
                if any((As_geq*dq).<tol_c)
                    maintain_vel_cond = all(abs.(dAs_eq*dq + As_eq*ddq).<tol_c) & all((dAs_geq*dq + As_geq*ddq).>0)
                end
            end

            if all(c_λ.>=0) && all(sep_vel_cond) && maintain_vel_cond
    
                if sum(m[1:n_contacts]) > max_cons
                    contactMode = m
                    max_cons = sum(m[1:n_contacts])
                end
            end
        end
    end
    
    return contactMode
end

compute_FA (generic function with 1 method)

In [13]:
# typeof(compute_FA([0.837466888555605, -0.6048929143967906, -2.5764126036346857, 0.9011419662349551, -0.30678893569095467, 1.4880622844845557],[5.083951321099527, 0.0894132959786084, -0.0447066479893042]))==Float64

In [14]:
function compute_IV(x)
    
    q = x[1:3]
    dq = x[4:6]
    
    a = compute_a(q)
    
    active_cs = abs.(a) .< tol_c
    inactive_cs = abs.(a) .> tol_c
    
    if sum(active_cs) == 0
        return zeros(Int, 4)
    end
    
    possibleModes = zeros(Bool, 0, size(modes,2))
    
    contactMode = zeros(n_contacts*2)
    
    for k = 1:size(modes,1)
        m_cs = modes[k, 1:n_contacts].==1
        if length(findall(z->z==true, m_cs[inactive_cs])) == 0
            possibleModes = [possibleModes; modes[k, :]']
        end
    end
    
    max_cons = 0
    
    for kk = 1:size(possibleModes, 1)
        
        m = possibleModes[kk,:]
        
        separate_cs = (m[1:n_contacts].!=1) .& active_cs
        _, A_separate, _ = contact_mode_constraints(x, [separate_cs; ones(n_contacts)]) 

        dq_p, p_hat = computeResetMap(x, m)
        
        
        c_p_hat = contact_force_constraints(p_hat, m)
        As_eq, As_geq, _, _ = sliding_velocity_constraints(x, m)
        
        if all(c_p_hat.>=0) && all(A_separate*dq_p.>=0) && all(abs.(As_eq*dq_p).<tol_c) && all(As_geq*dq_p.>0)
            if sum(m[1:n_contacts]) > max_cons
                contactMode = m
                max_cons = sum(m[1:n_contacts])
            end
        end
    end
    
    return contactMode
end

compute_IV (generic function with 1 method)

In [15]:
compute_IV([0.11740765013725207, 0.39999926760423715, -1.4647906679782115e-6, 0.28712284865750953, -0.35890463854696275, -0.7178084359434168])

4-element Vector{Int64}:
 1
 1
 1
 1

In [16]:
function mode(x, u_control)
    return compute_FA(x, u_control)
end

mode (generic function with 1 method)

In [17]:
function guard_conditions(x, contactMode, u_control)
    q = x[1:3]
    dq = x[4:6]
    
    a = compute_a(q)
    a[contactMode[1:n_contacts] .== 1] .= 0.0
    
    v_all = zeros(n_contacts)
    _, As_geq, _, _ = sliding_velocity_constraints(x, contactMode)
    v_all[1:size(As_geq,1)] = -As_geq*dq
    
    ddq, λ = solveEOM(x, contactMode, u_control)
    c_λ = contact_force_constraints(λ, contactMode)
    c_λ_all = zeros(3*n_contacts)
    c_λ_all[1:length(c_λ)] = c_λ
    
    c = [a; v_all; c_λ_all]
    
    dir = [-ones(Int,length(a)); ones(Int,length(v_all)); ones(Int,length(c_λ_all))]
    
    return c, dir
end

guard_conditions (generic function with 1 method)

## Wrappers for ODE solver

In [18]:
function dynamics!(dx, x, p, t)
    # p from integrator: (contact mode, controller, t_control, h_control, u_control)
    
    if debug == true
        println("Dynamics evalutation ", t)
    end
    q = x[1:3]
    dq = x[4:6]
    contactMode = p[1]
    controller = p[2]
    t_control = p[3][1]
    h_control = p[4]
    u_control = p[5]
    
    if t > (t_control + h_control)
        
        p[3] .= [Float64(t)]
        u_control = controller(x, contactMode)
        p[5] .= u_control
        # println("Control evaluation ", controller, " ", u_control)
    end
    
    ddq, λ = solveEOM(x, contactMode, u_control)
    
    # println(ddq)
    dx .= [dq; ddq]
    c_λ = contact_force_constraints(λ, contactMode)
    if(sum(c_λ.<0) > 0)
        new_contactMode = compute_FA(x, u_control)
        if (all(new_contactMode .==0))
            return
        end
        p[1] .= reshape(new_contactMode,size(p[1]))
        if debug == true
            println("New mode from constraints ", new_contactMode)
            println(λ)
        end
    end
end

dynamics! (generic function with 1 method)

In [19]:
function conditions(out, x, t, integrator)
    contactMode = integrator.p[1]
    u_control = integrator.p[5]
    c, dir = guard_conditions(x, contactMode, u_control)
    out .= c
end

function affect!(integrator, idx)
    contactMode = integrator.p[1]
    x = integrator.u
    u_control = integrator.p[5]
    c, dir = guard_conditions(x, contactMode, u_control)
    
    # only consider upcrossing forces and constraints values(FA comp)
    # forces
    if dir[idx] > 0
        new_contactMode = compute_FA(x, u_control)
        if (eltype(new_contactMode)==Float64)
            return
        end
        integrator.p[1] .= reshape(new_contactMode,size(integrator.p[1]))
        if debug == true
            println("New mode from FA: ", new_contactMode)
            println(x, u_control)
        end
    end
    # constraints
#     if dir[idx] < 0
#         new_contactMode = contactMode
#         new_contactMode[idx] = false
#     end
end

function affect_neg!(integrator, idx)
    contactMode = integrator.p[1]
    x = integrator.u
    u_control = integrator.p[5]
    c, dir = guard_conditions(x, contactMode, u_control)
    
    # only consider down crossing constraint value(IV comp)
    if dir[idx] < 0
        new_contactMode = compute_IV(x)
        if (all(new_contactMode .==0))
            return
        end
        dq_p, p_hat = computeResetMap(x, new_contactMode)
        integrator.u .= [x[1:3]; dq_p]
        integrator.p[1] .= reshape(new_contactMode,size(integrator.p[1]))
        
        if debug == true
            println("New mode from IV: ", new_contactMode)
            println(x)
        end

    end
end

affect_neg! (generic function with 1 method)

## goal specific conditions

In [20]:
# reference conditions
xref = [0;h/2;0;1;0;0]
uref = [μ*m*g;0]
nominal_mode = [1 1 1 1]

# pusher location
pusher_p = [-w/2;0]

2-element Vector{Float64}:
 -0.5
  0.0

## Basic controllers

In [21]:
function no_controller(x, contactMode)
    return zeros(3)
end

function force_controller(x, contactMode)
    f = zeros(3)
    f[1] = 2*μ*m*g
    return f
end

function pd_controller(x, contactMode)
    # for sliding velocity
    q = x[1:3]
    dq = x[4:6]
    k = 10
    d = 1
    eq = q - xref[1:3]
    eq[1] = 0
    f = -k.*eq .- d.*(dq - xref[4:6])
    
    # add sliding force compensation
    if abs(f[1]) > 1e-3
        f[1] += μ*m*g
    end
    return f
end

function pd_pusher_controller(x, contactMode)
    # for sliding velocity
    q = x[1:3]
    dq = x[4:6]
    
    k = 10
    d = 1
    
    A = [1 0; 0 1; -pusher_p[2] pusher_p[1]]
    
    eq = q - xref[1:3]
    eq[1] = 0
    fd = -k.*eq .- d.*(dq - xref[4:6])
    
    # add sliding force compensation
    if abs(fd[1]) > 1e-3
        fd[1] += μ*m*g
    end
    
    c = A\fd
    f = A*c
    return f
end
    

pd_pusher_controller (generic function with 1 method)

## LQR 
Push the box with a pusher (a point contact)

In [22]:
"""
    riccati(A,B,Q,R,Qf,N)

Use backward riccati recursion to solve the finite-horizon time-invariant LQR problem.
Returns vectors of the feedback gains `K` and cost-to-go matrices `P`, where `length(K) == N-1`,
`length(P) == N`, and `size(K[i]) == (m,n)` and `size(P[i]) == (n,n)`.

# Arguments:
* `A`: `(n,n)` discrete dynamics Jacobian wrt the state
* `B`: `(n,m)` discrete dynamics Jacobian wrt the control
* `Q`: `(n,n)` stage-wise cost matrix for states
* `R`: `(m,m)` stage-wise cost matrix for controls
* `Qf`: `(n,n)` cost matrix for terminal state
* `N`: Integer number of time steps (horizon length).
"""
function riccati(A,B,Q,R,Qf,N)
    # initialize the output
    n,m = size(B)
    P = [zeros(n,n) for k = 1:N]
    K = [zeros(m,n) for k = 1:N-1]
    
    P[end] .= Qf
    for k = reverse(1:N-1) 
        K[k] .= (R + B'P[k+1]*B)\(B'P[k+1]*A)
        P[k] .= Q + A'P[k+1]*A - A'P[k+1]*B*K[k]
    end
    
    # return the feedback gains and ctg matrices
    return K,P
end

riccati

In [23]:
function rk4_step(f,xk,u)
    # classic rk4 parameters
    a21 = 0.5
    a31 = 0
    a32 = 0.5
    a41 = 0
    a42 = 0
    a43 = 1
    b1 = 1/6
    b2 = 1/3
    b3 = 1/3
    b4 = 1/6
    
    f1 = f(xk,u)
    f2 = f(xk + Δt*a21*f1,u)
    f3 = f(xk + Δt*a31*f1 + Δt*a32*f2,u)
    f4 = f(xk + Δt*a41*f1 + Δt*a42*f2 + Δt*a43*f3,u)

    xn = xk + Δt*(b1*f1 + b2*f2 + b3*f3 + b4*f4)
    
    return xn
end

function continuous_dynamics(x, u, mode)
    q = x[1:3]
    dq = x[4:6]
    
    A_f, A, dA = contact_mode_constraints(x, mode)
    
    # contact constraints
    Jc = [1 0; 0 1; -pusher_p[2] pusher_p[1]]
    
    # compute EOM matrices
    N = [0; g; 0]
    C = zeros(3,3)
    Y = Jc*u
    
    blockMat = [M A_f'; A zeros(size(A,1),size(A_f',2))] 

    b = [Y-N-C*dq; -dA*dq]
    
    if rank(blockMat) < length(b)
        z =pinv(blockMat)*b
    else
        z = blockMat\b
    end

    ddq = z[1:3]
    
    return [dq;ddq]
end

function continuous_dynamics_differentiable(x, u, mode)
    q = x[1:3]
    dq = x[4:6]
    
    A_f, A, dA = contact_mode_constraints(x, mode)
    
    # contact constraints
    Jc = [1 0; 0 1; -pusher_p[2] pusher_p[1]]
    
    # compute EOM matrices
    N = [0; g; 0]
    C = zeros(3,3)
    Y = Jc*u
    
    blockMat = [M A_f'; A zeros(size(A,1),size(A_f',2))] 

    b = [Y-N-C*dq; -dA*dq]
    
    H = [zeros(3,size(blockMat,2)); zeros(size(blockMat,1)-3,3) 1e-6*I]
    z =(blockMat.+H)\b


    ddq = z[1:3]
    
    return [dq;ddq]
end

function discrete_dynamics(x, u, mode)
    xn = rk4_step((_x,_u)->continuous_dynamics_differentiable(_x,_u,mode),x,u)
    return xn
end

discrete_dynamics (generic function with 1 method)

In [24]:
# compute infinite horizon K
A_lqr = ForwardDiff.jacobian(_x->discrete_dynamics(_x, uref, nominal_mode), xref)
B_lqr = ForwardDiff.jacobian(_u->discrete_dynamics(xref, _u, nominal_mode), uref)
Q_lqr = Diagonal([0;1.0;1.0;1.0;0.1;1.0])
R_lqr = Diagonal([0.1;0.1])
Ks, _ = riccati(A_lqr,B_lqr,Q_lqr,R_lqr,Q_lqr,50)
K_lqr = Ks[1]

2×6 Matrix{Float64}:
 0.0   4.88068e-6  -5.22899e-6  2.58945   8.85062e-6  -1.5144e-5
 0.0  -1.25659e-5   1.53925e-5  1.29473  -2.16825e-5   3.69867e-5

In [25]:
function pusher_lqr_controller(x, contactMode)
    Jc = [1 0; 0 1; -pusher_p[2] pusher_p[1]]
    u = uref .- K_lqr*(x .- xref)
    f = Jc*u
    return f
end

pusher_lqr_controller (generic function with 1 method)

## Hybrid controller

### Hybrid system definition

In [26]:
# domain
# ineqs > 0; eqs = 0
function domain(x, contactMode)
    
    cs = contactMode[1:n_contacts] .== 1
    ss = contactMode[n_contacts+1:end]
    
    q = x[1:3]
    dq = x[4:6]
    
    # a(q) = or > 0
    
    a = compute_a(q)
    
    eqs_a = a[cs]
    ineqs_a = a[.~cs]
    
    # A_eq*dq = 0; A_geq*dq > 0
    
    A_eq, A_geq, _, _ = sliding_velocity_constraints(x, contactMode)
    
    eqs_A = A_eq*dq
    ineqs_A = A_geq*dq
    
    # separating contacts
    separate_cs = (cs.!=1) .& (abs.(a) .< tol_c)
    _, A_separate, _ = contact_mode_constraints(x, [separate_cs; ones(n_contacts)])
    
    ineqs_A_sep = A_separate*dq
    
    ineqs = [zeros(0); ineqs_a; ineqs_A]#; ineqs_A_sep]
    eqs = [zeros(0); eqs_a; eqs_A]
    
    return ineqs, eqs
end

domain (generic function with 1 method)

In [27]:
function guard_set(x, mode_from, mode_to)
    
    ineqs = zeros(0)
    eqs = zeros(0)
    
    q = x[1:3]
    dq = x[4:6]
    
    d_ineqs, d_eqs = domain(x, mode_from)
    # c: d_ineqs > 0, d_eqs = 0
    ineqs = [ineqs; d_ineqs]
    eqs = [eqs; d_eqs]
    
    cs_mode_to = mode_to[1:n_contacts].==1
    cs_mode_from = mode_from[1:n_contacts].==1
    
    A_f_new, A_new, dA_new = contact_mode_constraints(x, mode_to)
    
    # if there is new impact
    new_cs = (cs_mode_from.==false).& (cs_mode_to.==true)
    if sum(new_cs) > 0
            a_new = compute_a(x)
        # c: a_new[new_cs] .== 0
        eqs = [eqs; a_new[new_cs]]
        
        _, A_impact, _, = contact_mode_constraints(x, [new_cs; ones(n_contacts)])
        #c: A_impact*dq < 0
        ineqs = [ineqs; -A_impact*dq]
        
        p_hat = pinv(A_new*inv(M)*A_f_new')*A_new*dq
        #c: contact_force_constraints(p_hat, mode_to) > 0
        ineqs = [ineqs; contact_force_constraints(p_hat, mode_to)]
    end
    
    # if it is stick -> slide: do nothing, guard = domain
    # if it is slide -> stick: A_stick dq = 0
    ss_mode_to = mode_to[n_contacts+1:end]
    ss_mode_from = mode_from[n_contacts+1:end]
    ss_new_stick = (cs_mode_from.==true).& (cs_mode_to.==true) .& (ss_mode_to .== 0) .& (ss_mode_from .!=0)
    
    if sum(ss_new_stick) > 0    
        A_stick, _, _, _ = sliding_velocity_constraints(x, [ss_new_stick; zeros(n_contacts)])
        # c: A_stick*dq = 0
        eqs = [eqs; A_stick*dq]
    end
        
    return ineqs, eqs
end

guard_set (generic function with 1 method)

In [28]:
function jumpmap(x, mode_from, mode_to)
    
    q = x[1:3]
    dq = x[4:6]

    A_f, A, dA = contact_mode_constraints(x, mode_to)
    
    c = size(A, 1)
    #
    blockMat = [M A_f'; A zeros(size(A,1),size(A_f',2))] 
    
    b = [M*dq; zeros(c)]

    H = [zeros(3,size(blockMat,2)); zeros(size(blockMat,1)-3,3) 1e-6*I]
    z =(blockMat.+H)\b
        
    dq_p = z[1:3]
    p_hat = z[4:end]
    
    return [x[1:3]; dq_p]
end

jumpmap (generic function with 1 method)

In [29]:
function V(x)
    q = x[1:3] + x[4:6]
    dq = x[4:6]
    v = 0.5*([q;dq] .- xref)'*Q_lqr*([q;dq] .- xref)
    return v
end 

function dVdx(x)
    res = ForwardDiff.gradient(V, x)
    return res
end

dVdx (generic function with 1 method)

In [30]:
function hybrid_constraints_matrix(x, u_ref)
    # A*z + b>= 0
    # u = z[1:2]
    # β = z[3:175]
    # α = z[176:end]
    
    n = 2+173+19
    m = 19 
    A = zeros(m, n)
    b = zeros(m)

    # α = 10
    
    Vx = dVdx(x)
    Vv = V(x)
    
    n_modes = size(modes,1)
    
    β_idx = 1
    
    nominal_mode = modes[10,:] # [1 1 1 1]
    

    # mode dynamics
    
    for k = 1:n_modes
    
        m = modes[k,:]
        
        d_ineq, d_eq = domain(x, m)
        
        # TODO: debug this!!!
        if sum(d_ineq.<0)>0 || sum(abs.(d_eq) .> tol_c)>0
            b[k] = 0.1
            continue
        end
        
        n_ineq = size(d_ineq, 1)
        n_eq = size(d_eq, 1)
        
        dfdu = ForwardDiff.jacobian(_u->continuous_dynamics_differentiable(x, _u, m), u_ref)
        
        b[k] = -Vx'*continuous_dynamics_differentiable(x, u_ref, m)
        # α
        A[k, 175+k] = -Vv
        A[k, 1:2] = -Vx'*dfdu
        A[k, β_idx:β_idx+n_ineq-1] .= -d_ineq
        A[k, β_idx+n_ineq:β_idx+n_ineq+n_eq-1] .= -d_eq 
        A[k, β_idx+n_ineq+n_eq:β_idx+n_ineq+2*n_eq-1] .= d_eq 
        
        β_idx += n_ineq + 2*n_eq
        
    end
    
    nominal_mode = modes[10,:]
    
    for k = 1:(n_modes-1)
        
        mode_from = modes[k,:]
        
        d_ineq, d_eq = guard_set(x, mode_from, nominal_mode)
        
        if sum(d_ineq.<0)>0 || sum(abs.(d_eq) .> tol_c)>0
            b[k] = 0.1
            continue
        end
        
        n_ineq = size(d_ineq, 1)
        n_eq = size(d_eq, 1)
        
        xp = jumpmap(x, mode_from, nominal_mode)
        
        dfdu = ForwardDiff.jacobian(_u->continuous_dynamics_differentiable(xp, _u, nominal_mode), u_ref)
        
        b[n_modes+k] = -Vx'*continuous_dynamics_differentiable(xp, u_ref, nominal_mode)
        A[k, 175+n_modes+k] = -Vv
        A[n_modes+k, 1:2] = -Vx'*dfdu
        A[n_modes+k, β_idx:β_idx+n_ineq-1] .= -d_ineq
        A[n_modes+k, β_idx+n_ineq:β_idx+n_ineq+n_eq-1] .= -d_eq 
        A[n_modes+k, β_idx+n_ineq+n_eq:β_idx+n_ineq+2*n_eq-1] .= d_eq 
        
        β_idx += n_ineq + 2*n_eq
        
    end
    
    return A, b
    
end

hybrid_constraints_matrix (generic function with 1 method)

In [31]:

# z = Variable(194)
# A, b = hybrid_constraints_matrix(zeros(6), zeros(2))

# problem = minimize(sumsquares(z - [zeros(175); 10*ones(19)]))
# problem.constraints += A*z + b >= 0
# problem.constraints += z[3:end] >= 0
# problem.constraints += z[1:2] <= 20

# Convex.solve!(problem, () -> SCS.Optimizer(verbose=true))
# problem.status

### Interface with Convex.jl

In [32]:
# n_control_var = 2+173+19
# n_control_constraints = 19 # total number of constraints

### define the controller

In [33]:
function hybrid_reference_lqr(x, contactMode)
    Jc = [1 0; 0 1; -pusher_p[2] pusher_p[1]]
    u = uref .- K_lqr*(x .- xref)
    return u
end

hybrid_reference_lqr (generic function with 1 method)

In [34]:
function hybrid_controller(x, contactMode)
    
    u_ref = hybrid_controller_reference(x, contactMode) # from reference controller
    
    α_ref = 10

    z = Variable(194)
    A, b = hybrid_constraints_matrix(x, u_ref)
    
    μ_mnp = 0.8
    FC = [1 0; μ_mnp -1; μ_mnp 1]

    problem = minimize(sumsquares(z - [zeros(175); α_ref*ones(19)]))
    problem.constraints += A*z + b >= 0
    problem.constraints += z[3:end] >= 0
    #problem.constraints += FC*(z[1:2]+u_ref) >= 0
    problem.constraints += z[1:2] <= 20
    problem.constraints += z[1:2] >= -20


    Convex.solve!(problem, () -> SCS.Optimizer(verbose=false))
    
    z_sol = evaluate(z)
    if any(isnan.(z_sol)) || sum(sol) == Inf # infeasible
        z_sol = zeros(175)
#         println("Infeasible: ")
#         println(x)
#         println(contactMode)
    end
    
    u = z_sol[1:2] .+ u_ref
    
    # return space wrench
    Jc = [1 0; 0 1; -pusher_p[2] pusher_p[1]]
    f = Jc*u
    
    return f
    
end

hybrid_controller (generic function with 1 method)

In [35]:
# θ0 = -0.3
# x0 = [0;h/2 - sin(θ0)*w/2;θ0;0;0;0]
# initial_mode = mode(x0, controller)

# println(pusher_lqr_controller(x0, initial_mode))
# println(hybrid_controller(x0, initial_mode))

## simulate the sliding box 

In [36]:
# specify the controller
controller = hybrid_controller
hybrid_controller_reference = hybrid_reference_lqr

h_control = Δt

0.05

In [34]:
# initial condition

# # basic test
# x0 = [0;h/2;0;0;0;0]

# # lqr fail 1
# θ0 = -0.3
# x0 = [0;h/2 - sin(θ0)*w/2;θ0;0;0;0]

# # lqr success 2
# θ0 = -0.3
# x0 = [0;h/2 - sin(θ0)*w/2 + 0.2;θ0;0;0;0]

# # lqr fail 3
# θ0 = -0.5
# x0 = [0;h/2 - sin(θ0)*w/2;θ0;0;0;0]

# # lqr fail 4
# θ0 = -0.4
# x0 = [0;h/2 - sin(θ0)*w/2;θ0;0;0;0]

# lqr fail 5
θ0 = -0.4
x0 = [0;h/2 - sin(θ0)*w/2 + 0.5;θ0;0;0;0]

6-element Vector{Float64}:
  0.0
  1.0947091711543253
 -0.4
  0.0
  0.0
  0.0

In [125]:
debug = true
initial_mode = mode(x0, controller(x0))
# initial_mode = [1 1 0 0]
tspan = (0.0, 3.0)
callback_length = 5*n_contacts

prob = ODEProblem(dynamics!, x0, tspan, (initial_mode, controller, [0.0], h_control, zeros(3)))
cb = VectorContinuousCallback(conditions, affect!, affect_neg!, callback_length)
sol = solve(prob, Tsit5(); callback = cb, abstol=1e-15,reltol=1e-15, adaptive=false,dt=Δt)
# sol = solve(prob, Tsit5(); callback = cb, abstol=1e-9,reltol=1e-9, adaptive=false,dt=Δt)
# sol = solve(prob, Tsit5(); callback = cb, adaptive=false,dt=Δt)
println("Simulation status: ", sol.retcode)

Dynamics evalutation 0.0
Dynamics evalutation 0.00805
Dynamics evalutation 0.01635
Dynamics evalutation 0.045000000000000005
Dynamics evalutation 0.049001277045225486
Dynamics evalutation 0.05
Dynamics evalutation 0.05
Dynamics evalutation 0.058050000000000004
Dynamics evalutation 0.06635
Dynamics evalutation 0.095
Dynamics evalutation 0.09900127704522549
Dynamics evalutation 0.1
Dynamics evalutation 0.1
Dynamics evalutation 0.10805000000000001
Dynamics evalutation 0.11635000000000001
Dynamics evalutation 0.14500000000000002
Dynamics evalutation 0.14900127704522548
Dynamics evalutation 0.15000000000000002
Dynamics evalutation 0.15000000000000002
Dynamics evalutation 0.15805000000000002
Dynamics evalutation 0.16635000000000003
Dynamics evalutation 0.19500000000000003
Dynamics evalutation 0.1990012770452255
Dynamics evalutation 0.2
Dynamics evalutation 0.2
Dynamics evalutation 0.20805
Dynamics evalutation 0.21635000000000001
Dynamics evalutation 0.24500000000000002
Dynamics evalutation 0

Dynamics evalutation 1.569875110168612
Dynamics evalutation 1.569875110168612
Dynamics evalutation 1.5779251101686118
Dynamics evalutation 1.586225110168612
Dynamics evalutation 1.6148751101686118
Dynamics evalutation 1.6188763872138374
Dynamics evalutation 1.619875110168612
Dynamics evalutation 1.619875110168612
Dynamics evalutation 1.6279251101686119
Dynamics evalutation 1.636225110168612
Dynamics evalutation 1.664875110168612
Dynamics evalutation 1.6688763872138375
Dynamics evalutation 1.669875110168612
Dynamics evalutation 1.669875110168612
Dynamics evalutation 1.677925110168612
Dynamics evalutation 1.686225110168612
Dynamics evalutation 1.714875110168612
Dynamics evalutation 1.7188763872138375
Dynamics evalutation 1.719875110168612
Dynamics evalutation 1.719875110168612
Dynamics evalutation 1.727925110168612
Dynamics evalutation 1.7362251101686121
Dynamics evalutation 1.764875110168612
Dynamics evalutation 1.7688763872138376
Dynamics evalutation 1.769875110168612
Dynamics evalutat

In [103]:
function boxshape(q)
    p1 = q[1:2] + R_2D(q[3])*[w/2;h/2]
    p2 = q[1:2] + R_2D(q[3])*[w/2;-h/2]
    p3 = q[1:2] + R_2D(q[3])*[-w/2;-h/2]
    p4 = q[1:2] + R_2D(q[3])*[-w/2;h/2]
    pp = [p1 p2 p3 p4]
    return Shape(pp[1,:], pp[2,:])
end

boxshape (generic function with 1 method)

In [104]:
# animation 
n = length(sol.t)
# n = Int(floor(0.77/Δt))
x = zeros(n)
y = zeros(n)
θ = zeros(n)
for k = 1:n
    x[k] = sol.u[k][1]
    y[k] = sol.u[k][2]
    θ[k] = sol.u[k][3]
end
anim = @animate for i ∈ 1:n
    plot([-10,5],[0,0], lw = 2, c=:black, xlims=(-3,3), ylims=(-0.5,3))
    plot!(boxshape([x[i],y[i],θ[i]]), aspect_ratio=:equal, c=:gray, opacity=.5, legend=false)
end

Animation("/tmp/jl_0kk7s4", ["000001.png", "000002.png", "000003.png", "000004.png", "000005.png", "000006.png", "000007.png", "000008.png", "000009.png", "000010.png"  …  "000055.png", "000056.png", "000057.png", "000058.png", "000059.png", "000060.png", "000061.png", "000062.png", "000063.png", "000064.png"])

In [105]:
gif(anim, "anim.gif", fps = Int(floor(n/tspan[2])));

┌ Info: Saved animation to 
│   fn = /home/xianyi/study/16715robotsim/project/src/anim.gif
└ @ Plots /home/xianyi/.julia/packages/Plots/AJMX6/src/animation.jl:114


## Compute Contration Region (Monte-carlo)

In [37]:
function boxshape(q)
    p1 = q[1:2] + R_2D(q[3])*[w/2;h/2]
    p2 = q[1:2] + R_2D(q[3])*[w/2;-h/2]
    p3 = q[1:2] + R_2D(q[3])*[-w/2;-h/2]
    p4 = q[1:2] + R_2D(q[3])*[-w/2;h/2]
    pp = [p1 p2 p3 p4]
    return Shape(pp[1,:], pp[2,:])
end

boxshape (generic function with 1 method)

In [38]:
hybrid_controller_reference = hybrid_reference_lqr
tspan = (0.0, 4.0)
callback_length = 5*n_contacts
h_control = Δt

0.05

In [39]:
N = 31

31

In [40]:
y_l = h/2
y_u = h/2 + 0.8
θ_l = -pi/4
θ_u = pi/4
δy = (y_u - y_l)/(N-1)
δθ = (θ_u - θ_l)/(N-1)
println(δy," ", (180/pi)*δθ)

0.026666666666666672 3.0


In [41]:
ctrl = "lqr"

"lqr"

In [42]:
if ctrl == "lqr"
    print("lqr")
    controller = pusher_lqr_controller
    errs = zeros(N*N);
    initial_errs = zeros(N*N, 2);
else 
    print("hybrid")
    controller = hybrid_controller
    errs = zeros(N*N);
    initial_errs = zeros(N*N, 2);
end

lqr

961×2 Matrix{Float64}:
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 ⋮    
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0

In [85]:
controller

pusher_lqr_controller (generic function with 1 method)

In [86]:
debug = false

false

In [186]:
k = 1
q0 = zeros(N*N,2)
for ky = 0:(N-1), kθ = 0:(N-1)
    print("k ", k, "ky ", ky,"kt ", kθ)
    q0[k,:] = [δy*ky;θ_l + δθ*kθ]
    println(q0[k,:])
    k+=1
end

k 1ky 0kt 0[0.0, -0.7853981633974483]
k 2ky 0kt 1[0.0, -0.7330382858376184]
k 3ky 0kt 2[0.0, -0.6806784082777885]
k 4ky 0kt 3[0.0, -0.6283185307179586]
k 5ky 0kt 4[0.0, -0.5759586531581288]
k 6ky 0kt 5[0.0, -0.5235987755982989]
k 7ky 0kt 6[0.0, -0.47123889803846897]
k 8ky 0kt 7[0.0, -0.4188790204786391]
k 9ky 0kt 8[0.0, -0.3665191429188092]
k 10ky 0kt 9[0.0, -0.3141592653589793]
k 11ky 0kt 10[0.0, -0.26179938779914946]
k 12ky 0kt 11[0.0, -0.20943951023931962]
k 13ky 0kt 12[0.0, -0.15707963267948966]
k 14ky 0kt 13[0.0, -0.10471975511965981]
k 15ky 0kt 14[0.0, -0.05235987755982996]
k 16ky 0kt 15[0.0, 0.0]
k 17ky 0kt 16[0.0, 0.05235987755982985]
k 18ky 0kt 17[0.0, 0.1047197551196597]
k 19ky 0kt 18[0.0, 0.15707963267948966]
k 20ky 0kt 19[0.0, 0.2094395102393195]
k 21ky 0kt 20[0.0, 0.26179938779914935]
k 22ky 0kt 21[0.0, 0.3141592653589793]
k 23ky 0kt 22[0.0, 0.36651914291880905]
k 24ky 0kt 23[0.0, 0.418879020478639]
k 25ky 0kt 24[0.0, 0.47123889803846897]
k 26ky 0kt 25[0.0, 0.5235987755982

k 193ky 6kt 6[0.16000000000000003, -0.47123889803846897]
k 194ky 6kt 7[0.16000000000000003, -0.4188790204786391]
k 195ky 6kt 8[0.16000000000000003, -0.3665191429188092]
k 196ky 6kt 9[0.16000000000000003, -0.3141592653589793]
k 197ky 6kt 10[0.16000000000000003, -0.26179938779914946]
k 198ky 6kt 11[0.16000000000000003, -0.20943951023931962]
k 199ky 6kt 12[0.16000000000000003, -0.15707963267948966]
k 200ky 6kt 13[0.16000000000000003, -0.10471975511965981]
k 201ky 6kt 14[0.16000000000000003, -0.05235987755982996]
k 202ky 6kt 15[0.16000000000000003, 0.0]
k 203ky 6kt 16[0.16000000000000003, 0.05235987755982985]
k 204ky 6kt 17[0.16000000000000003, 0.1047197551196597]
k 205ky 6kt 18[0.16000000000000003, 0.15707963267948966]
k 206ky 6kt 19[0.16000000000000003, 0.2094395102393195]
k 207ky 6kt 20[0.16000000000000003, 0.26179938779914935]
k 208ky 6kt 21[0.16000000000000003, 0.3141592653589793]
k 209ky 6kt 22[0.16000000000000003, 0.36651914291880905]
k 210ky 6kt 23[0.16000000000000003, 0.4188790204

k 376ky 12kt 3[0.32000000000000006, -0.6283185307179586]
k 377ky 12kt 4[0.32000000000000006, -0.5759586531581288]
k 378ky 12kt 5[0.32000000000000006, -0.5235987755982989]
k 379ky 12kt 6[0.32000000000000006, -0.47123889803846897]
k 380ky 12kt 7[0.32000000000000006, -0.4188790204786391]
k 381ky 12kt 8[0.32000000000000006, -0.3665191429188092]
k 382ky 12kt 9[0.32000000000000006, -0.3141592653589793]
k 383ky 12kt 10[0.32000000000000006, -0.26179938779914946]
k 384ky 12kt 11[0.32000000000000006, -0.20943951023931962]
k 385ky 12kt 12[0.32000000000000006, -0.15707963267948966]
k 386ky 12kt 13[0.32000000000000006, -0.10471975511965981]
k 387ky 12kt 14[0.32000000000000006, -0.05235987755982996]
k 388ky 12kt 15[0.32000000000000006, 0.0]
k 389ky 12kt 16[0.32000000000000006, 0.05235987755982985]
k 390ky 12kt 17[0.32000000000000006, 0.1047197551196597]
k 391ky 12kt 18[0.32000000000000006, 0.15707963267948966]
k 392ky 12kt 19[0.32000000000000006, 0.2094395102393195]
k 393ky 12kt 20[0.320000000000000

k 558ky 17kt 30[0.4533333333333334, 0.7853981633974483]
k 559ky 18kt 0[0.4800000000000001, -0.7853981633974483]
k 560ky 18kt 1[0.4800000000000001, -0.7330382858376184]
k 561ky 18kt 2[0.4800000000000001, -0.6806784082777885]
k 562ky 18kt 3[0.4800000000000001, -0.6283185307179586]
k 563ky 18kt 4[0.4800000000000001, -0.5759586531581288]
k 564ky 18kt 5[0.4800000000000001, -0.5235987755982989]
k 565ky 18kt 6[0.4800000000000001, -0.47123889803846897]
k 566ky 18kt 7[0.4800000000000001, -0.4188790204786391]
k 567ky 18kt 8[0.4800000000000001, -0.3665191429188092]
k 568ky 18kt 9[0.4800000000000001, -0.3141592653589793]
k 569ky 18kt 10[0.4800000000000001, -0.26179938779914946]
k 570ky 18kt 11[0.4800000000000001, -0.20943951023931962]
k 571ky 18kt 12[0.4800000000000001, -0.15707963267948966]
k 572ky 18kt 13[0.4800000000000001, -0.10471975511965981]
k 573ky 18kt 14[0.4800000000000001, -0.05235987755982996]
k 574ky 18kt 15[0.4800000000000001, 0.0]
k 575ky 18kt 16[0.4800000000000001, 0.05235987755982

k 748ky 24kt 3[0.6400000000000001, -0.6283185307179586]
k 749ky 24kt 4[0.6400000000000001, -0.5759586531581288]
k 750ky 24kt 5[0.6400000000000001, -0.5235987755982989]
k 751ky 24kt 6[0.6400000000000001, -0.47123889803846897]
k 752ky 24kt 7[0.6400000000000001, -0.4188790204786391]
k 753ky 24kt 8[0.6400000000000001, -0.3665191429188092]
k 754ky 24kt 9[0.6400000000000001, -0.3141592653589793]
k 755ky 24kt 10[0.6400000000000001, -0.26179938779914946]
k 756ky 24kt 11[0.6400000000000001, -0.20943951023931962]
k 757ky 24kt 12[0.6400000000000001, -0.15707963267948966]
k 758ky 24kt 13[0.6400000000000001, -0.10471975511965981]
k 759ky 24kt 14[0.6400000000000001, -0.05235987755982996]
k 760ky 24kt 15[0.6400000000000001, 0.0]
k 761ky 24kt 16[0.6400000000000001, 0.05235987755982985]
k 762ky 24kt 17[0.6400000000000001, 0.1047197551196597]
k 763ky 24kt 18[0.6400000000000001, 0.15707963267948966]
k 764ky 24kt 19[0.6400000000000001, 0.2094395102393195]
k 765ky 24kt 20[0.6400000000000001, 0.261799387799

k 931ky 30kt 0[0.8000000000000002, -0.7853981633974483]
k 932ky 30kt 1[0.8000000000000002, -0.7330382858376184]
k 933ky 30kt 2[0.8000000000000002, -0.6806784082777885]
k 934ky 30kt 3[0.8000000000000002, -0.6283185307179586]
k 935ky 30kt 4[0.8000000000000002, -0.5759586531581288]
k 936ky 30kt 5[0.8000000000000002, -0.5235987755982989]
k 937ky 30kt 6[0.8000000000000002, -0.47123889803846897]
k 938ky 30kt 7[0.8000000000000002, -0.4188790204786391]
k 939ky 30kt 8[0.8000000000000002, -0.3665191429188092]
k 940ky 30kt 9[0.8000000000000002, -0.3141592653589793]
k 941ky 30kt 10[0.8000000000000002, -0.26179938779914946]
k 942ky 30kt 11[0.8000000000000002, -0.20943951023931962]
k 943ky 30kt 12[0.8000000000000002, -0.15707963267948966]
k 944ky 30kt 13[0.8000000000000002, -0.10471975511965981]
k 945ky 30kt 14[0.8000000000000002, -0.05235987755982996]
k 946ky 30kt 15[0.8000000000000002, 0.0]
k 947ky 30kt 16[0.8000000000000002, 0.05235987755982985]
k 948ky 30kt 17[0.8000000000000002, 0.1047197551196

In [87]:
errs

961-element Vector{Float64}:
 3.2418514695262193
 3.2418512493511806
 3.241851076286848
 3.2418509599065297
 3.2418508168816724
 3.2418507421532174
 3.241850655193741
 3.2418520191116653
 7.327783627291089e-6
 5.085207397640682e-6
 4.1637550909495885e-6
 4.278136840898357e-6
 3.6740404457299797e-6
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [110]:
k = 1
# t_sim_start = time()

for ky = 0:(N-1), kθ = 0:(N-1)
    
    if k<=789 || k > 800
        k+=1
        continue
    end
#     if kθ > 6 && kθ < 14 || kθ == 2 || kθ == 3
#         errs[k] = NaN
#         k+=1
#         continue
#     end
    θ0 = θ_l + δθ*kθ
    y0 = δy*ky + sin(abs(θ0) + atan(h/w))*sqrt(h^2 + w^2)/2
    x0 = [0;y0;θ0;0;0;0]
    initial_mode = mode(x0, zeros(3))
    
    
    initial_errs[k,:] .= [y0; θ0]
#     t_sim_start = time()
    prob = ODEProblem(dynamics!, x0, (0,2.5), (initial_mode, controller, [0.0], h_control, zeros(3)))
    cb = VectorContinuousCallback(conditions, affect!, affect_neg!, callback_length)
    sol = solve(prob, Tsit5(); callback = cb, adaptive=false, dt=Δt)
    
    print("Sample ", k, " y0 ", y0, ", θ0 ", θ0, ", Simulation status: ", sol.retcode)
     
    u_end = sol.u[end]
    err = norm(u_end[2:4] .- xref[2:4])
    println(", err: ", err)
    errs[k] = err
    
    if string(sol.retcode) != "Success"
        k+=1
        continue
    end
    
    # animation 
    n_plot = length(sol.t)
    # n = Int(floor(0.77/Δt))
    x_plot = zeros(n_plot)
    y_plot = zeros(n_plot)
    θ_plot = zeros(n_plot)
    for k = 1:n_plot
        x_plot[k] = sol.u[k][1]
        y_plot[k] = sol.u[k][2]
        θ_plot[k] = sol.u[k][3]
    end

    anim = @animate for i ∈ 1:n_plot
        plot([-10,5],[0,0], lw = 2, c=:black, xlims=(-2,3), ylims=(-0.5,3))
        plot!(boxshape([x_plot[i],y_plot[i],θ_plot[i]]), aspect_ratio=:equal, c=:gray, opacity=.5, legend=false)
    end
    
    gif(anim, "figs_sample/" *ctrl* "_" * string(k) * ".gif", fps = Int(floor(n_plot/tspan[2])));
    
    k += 1
    
    if (mod(k, 10) == 0)
        save(ctrl*"_errs.jld", "errs", errs)
    end
    
end

save(ctrl*"_errs.jld", "errs", errs)


Sample 790 y0 1.0922864586899683, θ0 -0.05235987755982996, Simulation status: Success, err: 4.508437171486888
Sample 791 y0 1.0666666666666669, θ0 0.0, Simulation status: Unstable, err: NaN
Sample 792 y0 1.0922864586899683, θ0 0.05235987755982985, Simulation status: Unstable, err: NaN
Sample 793 y0 1.1167396564478027, θ0 0.1047197551196597, Simulation status: Success, err: 4.381499143411489
Sample 794 y0 1.1399592354248373, θ0 0.15707963267948966, Simulation status: Unstable, err: NaN
Sample 795 y0 1.1618815523690686, θ0 0.2094395102393195, Simulation status: Success, err: 0.00017409831951103246
Sample 796 y0 1.1824465197335545, θ0 0.26179938779914935, Simulation status: Success, err: 0.00011768190587243903
Sample 797 y0 1.201597770372202, θ0 0.3141592653589793, Simulation status: Success, err: 4.7743776264026285e-5
Sample 798 y0 1.2192828120381978, θ0 0.36651914291880905, Simulation status: Success, err: 0.0002869967778731314
Sample 799 y0 1.2354531712616073, θ0 0.418879020478639, Sim

┌ Info: Saved animation to 
│   fn = /home/xianyi/study/16715robotsim/project/src/figs_sample/lqr_790.gif
└ @ Plots /home/xianyi/.julia/packages/Plots/AJMX6/src/animation.jl:114
┌ Warning: Instability detected. Aborting
└ @ SciMLBase /home/xianyi/.julia/packages/SciMLBase/7GnZA/src/integrator_interface.jl:351
┌ Warning: Instability detected. Aborting
└ @ SciMLBase /home/xianyi/.julia/packages/SciMLBase/7GnZA/src/integrator_interface.jl:351
┌ Info: Saved animation to 
│   fn = /home/xianyi/study/16715robotsim/project/src/figs_sample/lqr_793.gif
└ @ Plots /home/xianyi/.julia/packages/Plots/AJMX6/src/animation.jl:114
┌ Warning: Instability detected. Aborting
└ @ SciMLBase /home/xianyi/.julia/packages/SciMLBase/7GnZA/src/integrator_interface.jl:351
┌ Info: Saved animation to 
│   fn = /home/xianyi/study/16715robotsim/project/src/figs_sample/lqr_795.gif
└ @ Plots /home/xianyi/.julia/packages/Plots/AJMX6/src/animation.jl:114
┌ Info: Saved animation to 
│   fn = /home/xianyi/study/16715robots

In [118]:
save(ctrl*"_errs.jld", "errs", errs)


In [193]:
hybrid_errs = load("hybrid_errs.jld","errs");
lqr_errs = load("lqr_errs.jld", "errs");
hybrid_errs[hybrid_errs.==0.0] .= NaN
lqr_errs[lqr_errs.==0.0] .= NaN

20-element view(::Vector{Float64}, [253, 254, 264, 287, 293, 384, 385, 390, 421, 591, 592, 605, 639, 697, 745, 760, 789, 823, 850, 916]) with eltype Float64:
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN

In [199]:
hybrid_idx_success = (hybrid_errs.<= 0.2)
hybrid_idx_fail = hybrid_errs .> 0.2
hybrid_idx_nan = isnan.(hybrid_errs)

scatter(q0[hybrid_idx_success,1], q0[hybrid_idx_success,2],m=:circle, c=:green,markersize=2, markerstrokewidth=0,
        xlabel = "Δy", ylabel = "Δθ", label = "success")
scatter!(q0[hybrid_idx_fail,1], q0[hybrid_idx_fail,2],m=:circle, c=:red,markersize=2, markerstrokewidth=0, label = "fail")
scatter!(q0[hybrid_idx_nan,1], q0[hybrid_idx_nan,2],m=:circle, c=:gray,markersize=2, markerstrokewidth=0, label = "nan")
savefig("hybrid_trials.pdf")

In [200]:
lqr_idx_success = (lqr_errs.<= 0.2)
lqr_idx_fail = lqr_errs .> 0.2
lqr_idx_nan = isnan.(hybrid_errs)

scatter(q0[lqr_idx_success,1], q0[lqr_idx_success,2],m=:circle, c=:green,markersize=2, markerstrokewidth=0,
        xlabel = "Δy", ylabel = "Δθ", label = "success")
scatter!(q0[lqr_idx_fail,1], q0[lqr_idx_fail,2],m=:circle, c=:red,markersize=2, markerstrokewidth=0, label = "fail")
scatter!(q0[lqr_idx_nan,1], q0[lqr_idx_nan,2],m=:circle, c=:gray,markersize=2, markerstrokewidth=0, label = "nan")
savefig("lqr_trials.pdf")

In [191]:
errs[errs.==0.0] .= NaN

0-element view(::Vector{Float64}, Int64[]) with eltype Float64

In [178]:
sum(errs[528:558].==0.0)

2

In [165]:
println((errs[528:558].<0.2) .& (errs[528:558].> 0))

Bool[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [81]:
function check_unstable(dt,u,p,t)
    return any(isnan,u) | any(isinf,u)
end

check_unstable (generic function with 1 method)

## debug

In [84]:
debug = true
kk = 823
θ0 = initial_errs[kk, 2]
y0 = initial_errs[kk, 1]
x0 = [0;y0;θ0;0;0;0]
# x0 = [0; h;0;0;0;0]


initial_mode = mode(x0, zeros(3))

prob = ODEProblem(dynamics!, x0, tspan, (initial_mode, controller, [0.0], h_control, zeros(3)))
cb = VectorContinuousCallback(conditions, affect!, affect_neg!, callback_length)
sol = solve(prob, Tsit5(); callback = cb, abstol=1e-15,reltol=1e-15, dt=Δt, unstable_check=check_unstable)

# n_plot = length(sol.t)
# # n = Int(floor(0.77/Δt))
# x_plot = zeros(n_plot)
# y_plot = zeros(n_plot)
# θ_plot = zeros(n_plot)
# for k = 1:n_plot
#     x_plot[k] = sol.u[k][1]
#     y_plot[k] = sol.u[k][2]
#     θ_plot[k] = sol.u[k][3]
# end

# anim = @animate for i ∈ 1:n_plot
#     plot([-10,5],[0,0], lw = 2, c=:black, xlims=(-2,3), ylims=(-0.5,3))
#     plot!(boxshape([x_plot[i],y_plot[i],θ_plot[i]]), aspect_ratio=:equal, c=:gray, opacity=.5, legend=false)
# end

# gif(anim, "anim.gif", fps = Int(floor(n_plot/tspan[2])));
    

Dynamics evalutation 0.0
Dynamics evalutation 0.00805
Dynamics evalutation 0.01635
Dynamics evalutation 0.045000000000000005
Dynamics evalutation 0.049001277045225486
Dynamics evalutation 0.05
Dynamics evalutation 0.05
Dynamics evalutation 0.056535745383288025
Dynamics evalutation 0.06327446422568436
Dynamics evalutation 0.08653522263949825
Dynamics evalutation 0.08978383481037885
Dynamics evalutation 0.09059469182166473
Dynamics evalutation 0.09059469182166473
Dynamics evalutation 0.05130714907665761
Dynamics evalutation 0.05265489284513688
Dynamics evalutation 0.057307044527899656
Dynamics evalutation 0.05795676696207577
Dynamics evalutation 0.05811893836433295
Dynamics evalutation 0.05811893836433295
Dynamics evalutation 0.05026142981533152
Dynamics evalutation 0.05053097856902738
Dynamics evalutation 0.05146140890557993
Dynamics evalutation 0.051591353392415155
Dynamics evalutation 0.05162378767286659
Dynamics evalutation 0.05162378767286659
Dynamics evalutation 0.05005228596306631

Dynamics evalutation 0.12069305575490002
Dynamics evalutation 0.12107495125279394
Dynamics evalutation 0.12107495125279394
Dynamics evalutation 0.102571398235672
Dynamics evalutation 0.103206155375751
Dynamics evalutation 0.10539721466048149
Dynamics evalutation 0.10570321938022739
Dynamics evalutation 0.10577959847980618
Dynamics evalutation 0.10577959847980618
Dynamics evalutation 0.1020788878763818
Dynamics evalutation 0.1022058393043976
Dynamics evalutation 0.10264405116134369
Dynamics evalutation 0.10270525210529287
Dynamics evalutation 0.10272052792520862
Dynamics evalutation 0.10272052792520862
Dynamics evalutation 0.10198038580452375
Dynamics evalutation 0.10200577609012691
Dynamics evalutation 0.10209341846151614
Dynamics evalutation 0.10210565865030596
Dynamics evalutation 0.10210871381428911
Dynamics evalutation 0.10210871381428911
Dynamics evalutation 0.10196068539015214
Dynamics evalutation 0.10196576344727278
Dynamics evalutation 0.10198329192155062
Dynamics evalutation 0

Dynamics evalutation 0.1082420130454688
Dynamics evalutation 0.10830663349174251
Dynamics evalutation 0.10830663349174251
Dynamics evalutation 0.10921561143326504
Dynamics evalutation 0.11015281850340629
Dynamics evalutation 0.1133878766431107
Dynamics evalutation 0.11383968690050578
Dynamics evalutation 0.11395245921548494
Dynamics evalutation 0.11395245921548494
Dynamics evalutation 0.11539535058293504
Dynamics evalutation 0.11688305224129973
Dynamics evalutation 0.12201831158011281
Dynamics evalutation 0.12273550513381866
Dynamics evalutation 0.1229145173984048
Dynamics evalutation 0.1229145173984048
Dynamics evalutation 0.12481712310615474
Dynamics evalutation 0.12677881594768572
Dynamics evalutation 0.13355020147899455
Dynamics evalutation 0.13449589744725973
Dynamics evalutation 0.13473194415461562
Dynamics evalutation 0.13473194415461562
Dynamics evalutation 0.1374574687887556
Dynamics evalutation 0.14026763704507378
Dynamics evalutation 0.14996779614670233
Dynamics evalutation 

Dynamics evalutation 0.15166253396184304
Dynamics evalutation 0.1516626660584703
Dynamics evalutation 0.1516626990299273
Dynamics evalutation 0.1516626990299273
Dynamics evalutation 0.15166377559639238
Dynamics evalutation 0.15166488559659863
Dynamics evalutation 0.15166871710333468
Dynamics evalutation 0.1516692522139787
Dynamics evalutation 0.15166938577815772
Dynamics evalutation 0.15166938577815772
Dynamics evalutation 0.1516729867754362
Dynamics evalutation 0.15167669960492827
Dynamics evalutation 0.15168951557660879
Dynamics evalutation 0.15169130546328585
Dynamics evalutation 0.15169175222088113
Dynamics evalutation 0.15169175222088113
Dynamics evalutation 0.15170191226203472
Dynamics evalutation 0.1517123878324167
Dynamics evalutation 0.15174854748198818
Dynamics evalutation 0.1517535975614225
Dynamics evalutation 0.15175485806655561
Dynamics evalutation 0.15175485806655561
Dynamics evalutation 0.15177928870482155
Dynamics evalutation 0.1518044780585616
Dynamics evalutation 0.1

Dynamics evalutation 0.20306687372667268
Dynamics evalutation 0.20306687372672944
Dynamics evalutation 0.2030668737267436
Dynamics evalutation 0.2030668737267436
Dynamics evalutation 0.2030668737278854
Dynamics evalutation 0.20306687372906265
Dynamics evalutation 0.20306687373312635
Dynamics evalutation 0.20306687373369386
Dynamics evalutation 0.20306687373383553
Dynamics evalutation 0.20306687373383553
Dynamics evalutation 0.20306687374525356
Dynamics evalutation 0.20306687375702617
Dynamics evalutation 0.20306687379766303
Dynamics evalutation 0.20306687380333838
Dynamics evalutation 0.20306687380475497
Dynamics evalutation 0.20306687380475497
Dynamics evalutation 0.20306687391893527
Dynamics evalutation 0.20306687403666152
Dynamics evalutation 0.2030668744430299
Dynamics evalutation 0.20306687449978356
Dynamics evalutation 0.20306687451394934
Dynamics evalutation 0.20306687451394934
Dynamics evalutation 0.2030668756557523
Dynamics evalutation 0.20306687683301494
Dynamics evalutation 

...

Dynamics evalutation 0.8717545604990151
Dynamics evalutation 0.8717845753746218
Dynamics evalutation 0.8718881809392158
Dynamics evalutation 0.8719026505576231
Dynamics evalutation 0.8719062621895812
Dynamics evalutation 0.8719062621895812
Dynamics evalutation 0.8719370432714931
Dynamics evalutation 0.8719687802876258
Dynamics evalutation 0.8720783303493371
Dynamics evalutation 0.8720936301799559
Dynamics evalutation 0.8720974490337544
Dynamics evalutation 0.8720974490337544
Dynamics evalutation 0.8721296345586752
Dynamics evalutation 0.8721628196340594
Dynamics evalutation 0.8722773681171627
Dynamics evalutation 0.8722933660304614
Dynamics evalutation 0.8722973591264302
Dynamics evalutation 0.8722973591264302
Dynamics evalutation 0.872330700099148
Dynamics evalutation 0.8723650765058012
Dynamics evalutation 0.8724837372347907
Dynamics evalutation 0.8725003094669399
Dynamics evalutation 0.8725044459134974
Dynamics evalutation 0.8725044459134974
Dynamics evalutation 0.8725388074920399
D

Dynamics evalutation 0.8822932005661244
Dynamics evalutation 0.8823377934141946
Dynamics evalutation 0.8823837711333229
Dynamics evalutation 0.8825424773565791
Dynamics evalutation 0.8825646423676803
Dynamics evalutation 0.8825701747777407
Dynamics evalutation 0.8825701747777407
Dynamics evalutation 0.8826149990825228
Dynamics evalutation 0.8826612154464597
Dynamics evalutation 0.8828207454255911
Dynamics evalutation 0.8828430254829566
Dynamics evalutation 0.8828485866086855
Dynamics evalutation 0.8828485866086855
Dynamics evalutation 0.8828936562629722
Dynamics evalutation 0.8829401255959635
Dynamics evalutation 0.8831005287755057
Dynamics evalutation 0.8831229307845906
Dynamics evalutation 0.8831285223495968
Dynamics evalutation 0.8831285223495968
Dynamics evalutation 0.8831738325533071
Dynamics evalutation 0.8832205499062009
Dynamics evalutation 0.8833818092026356
Dynamics evalutation 0.8834043307775451
Dynamics evalutation 0.8834099521863067
Dynamics evalutation 0.8834099521863067


Dynamics evalutation 0.89976010506774
Dynamics evalutation 0.8998294016390533
Dynamics evalutation 0.8999008502777988
Dynamics evalutation 0.9001474772055165
Dynamics evalutation 0.9001819212775797
Dynamics evalutation 0.9001905185541582
Dynamics evalutation 0.9001905185541582
Dynamics evalutation 0.9002603310916124
Dynamics evalutation 0.9003323117202919
Dynamics evalutation 0.9005807749747098
Dynamics evalutation 0.9006154755093165
Dynamics evalutation 0.9006241367992155
Dynamics evalutation 0.9006241367992155
Dynamics evalutation 0.900694382166832
Dynamics evalutation 0.9007668090676041
Dynamics evalutation 0.9010168127672576
Dynamics evalutation 0.9010517284414163
Dynamics evalutation 0.9010604434303734
Dynamics evalutation 0.9010604434303734
Dynamics evalutation 0.9011310202180512
Dynamics evalutation 0.9012037888314333
Dynamics evalutation 0.9014549720571439
Dynamics evalutation 0.9014900524646545
Dynamics evalutation 0.9014988085712294
Dynamics evalutation 0.9014988085712294
Dyn

Excessive output truncated after 524300 bytes.


Dynamics evalutation 0.9212472708350814
Dynamics evalutation 0.9212472708415346
Dynamics evalutation 0.9212472708638093
Dynamics evalutation 0.9212472708669203
Dynamics evalutation 0.9212472708676968
Dynamics evalutation 0.9212472708676968
Dynamics evalutation 0.9212472708326698
Dynamics evalutation 0.9212472708366364
Dynamics evalutation 0.9212472708503281
Dynamics evalutation 0.9212472708522403
Dynamics evalutation 0.9212472708527176
Dynamics evalutation 0.9212472708527176
Dynamics evalutation 0.9212472708313542
Dynamics evalutation 0.9212472708339642
Dynamics evalutation 0.9212472708429735
Dynamics evalutation 0.9212472708442319
Dynamics evalutation 0.9212472708445459
Dynamics evalutation 0.9212472708445459
Dynamics evalutation 0.921247270830589
Dynamics evalutation 0.92124727083241
Dynamics evalutation 0.921247270838696
Dynamics evalutation 0.9212472708395739
Dynamics evalutation 0.9212472708397931
Dynamics evalutation 0.9212472708397931
Dynamics evalutation 0.9212472708301187
Dyn

In [80]:
sol

retcode: Unstable
Interpolation: specialized 4th order "free" interpolation
t: 1-element Vector{Float64}:
 0.0
u: 1-element Vector{Vector{Float64}}:
 [0.0, 1.118953125356635, 0.05235987755982985, 0.0, 0.0, 0.0]

false